In [ ]:
# Enable outputting results interactively. 
# All the results from code in a given cell will be displayed instead of only the last one.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
import pickle
import numpy as np
import sqlite3
import math
import datetime

from re import sub
from decimal import Decimal

import re

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
calendar =  pickle.load( open( "../data/all_SF_calendar.p", "rb" ) )
calendar.shape
# 

# limit to listings where the min. nights is less than 30 days. listings with 30 days + requirement are exempt from the regulation.

# calendar = calendar[calendar['minimum_nights'] <30 ]
# calendar.shape

In [4]:
calendar['price'] = calendar['price'].str.replace("\$|,", "").astype(float)
calendar.dtypes

listing_id                  int64
date               datetime64[ns]
available                  object
price                     float64
adjusted_price             object
minimum_nights            float64
maximum_nights            float64
file_scraped_dt    datetime64[ns]
lead_time                   int64
dtype: object

In [5]:
calendar = calendar.set_index(pd.DatetimeIndex(calendar['file_scraped_dt']))
calendar['month'] = calendar.index.strftime('%Y-%m')
calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,file_scraped_dt,lead_time,month
file_scraped_dt,,,,,,,,,,
2019-06-02,75422,2019-08-16,t,40.0,$40.00,30.0,1825.0,2019-06-02,75,2019-06
2019-06-02,75422,2019-08-17,t,40.0,$40.00,30.0,1825.0,2019-06-02,76,2019-06
2019-06-02,75422,2019-08-18,t,40.0,$40.00,30.0,1825.0,2019-06-02,77,2019-06
2019-06-02,75422,2019-08-19,t,40.0,$40.00,30.0,1825.0,2019-06-02,78,2019-06
2019-06-02,75422,2019-08-20,t,40.0,$40.00,30.0,1825.0,2019-06-02,79,2019-06


In [37]:
n_bookings =  pickle.load( open( "../data/ls_booking.p", "rb" ) )
n_bookings.head()
n_bookings.shape

,listing_id,month,counts
0,958,2009-07,2
1,958,2009-08,1
2,958,2009-09,1
3,958,2009-11,1
4,958,2010-02,2


(211424, 3)

In [51]:
ls = n_bookings.groupby('listing_id')['counts'].sum().reset_index(name = 'counts').sort_values(by='counts', ascending = False)

ls.head()
ls_sub = ls[ls['counts'] >= 10]
ls_sub = ls_sub.drop('counts', axis = 1)
ls_sub.head()

,listing_id,counts
959,545685,741
12,8739,715
413,205842,674
3049,1920033,672
1022,585326,667


,listing_id
959,545685
12,8739
413,205842
3049,1920033
1022,585326


In [52]:
bookings_sub = n_bookings.merge(ls_sub, how = 'inner', on = 'listing_id')
bookings_sub = bookings_sub.drop('counts', axis = 1)
bookings_sub.head()
bookings_sub.shape

,listing_id,month
0,958,2009-07
1,958,2009-08
2,958,2009-09
3,958,2009-11
4,958,2010-02


(185548, 2)

In [53]:
calendar_sub = calendar.merge(bookings_sub, how = 'inner', on = ['listing_id', 'month'] )
calendar_sub.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,file_scraped_dt,lead_time,month
0,9225,2019-06-04,t,105.0,$105.00,1.0,365.0,2019-06-02,2,2019-06
1,9225,2019-06-09,t,105.0,$105.00,1.0,365.0,2019-06-02,7,2019-06
2,9225,2019-06-13,t,115.0,$115.00,1.0,365.0,2019-06-02,11,2019-06
3,9225,2019-06-19,t,150.0,$150.00,1.0,365.0,2019-06-02,17,2019-06
4,9225,2019-06-20,t,150.0,$150.00,1.0,365.0,2019-06-02,18,2019-06


In [54]:
calendar_sub.shape[0]/calendar.shape[0]

0.4158734776125262

In [55]:
pickle.dump(calendar_sub, open('../data/calendar_pricing.p','wb'))

In [61]:
# test = calendar[calendar['listing_id'] == 8739].sort_values(by= ['date', 'lead_time'])
# test['2018-08-06']


test = calendar[calendar['date'] == '2018-09-26']

In [63]:
test

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,file_scraped_dt,lead_time,month
file_scraped_dt,,,,,,,,,,
2018-09-08,26488,2018-09-26,t,1000.0,NaN,NaN,NaN,2018-09-08,18,2018-09
2018-09-08,5858,2018-09-26,t,235.0,NaN,NaN,NaN,2018-09-08,18,2018-09
2018-09-08,7918,2018-09-26,t,62.0,NaN,NaN,NaN,2018-09-08,18,2018-09
2018-09-08,8142,2018-09-26,t,65.0,NaN,NaN,NaN,2018-09-08,18,2018-09
2018-09-08,32027,2018-09-26,t,600.0,NaN,NaN,NaN,2018-09-08,18,2018-09
2018-09-08,105497,2018-09-26,t,77.0,NaN,NaN,NaN,2018-09-08,18,2018-09
2018-09-08,38607,2018-09-26,t,139.0,NaN,NaN,NaN,2018-09-08,18,2018-09
2018-09-08,39174,2018-09-26,t,107.0,NaN,NaN,NaN,2018-09-08,18,2018-09
2018-09-08,108535,2018-09-26,t,120.0,NaN,NaN,NaN,2018-09-08,18,2018-09
